In [75]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [76]:
transform = transforms.Compose([transforms.ToTensor()])

mnist_train = datasets.MNIST(root='data/',download=False, train=True, transform=transform)
mnist_test = datasets.MNIST(root='data/', download=False, train=False, transform=transform)


In [77]:
#Loading dataset into dataloader
batch_size = 200
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)


In [78]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
     
        self.encoder = torch.nn.Sequential(
            nn.Linear(28*28, 500,bias=True),
            nn.ReLU(True),
            nn.Linear(500, 100,bias=True),
            nn.ReLU(True)
        )
        
    
        self.decoder = torch.nn.Sequential(
            nn.Linear(100, 500,bias=True),
            nn.ReLU(True),
            nn.Linear(500, 28*28,bias=True),
            nn.ReLU(True)
        )


    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        
        return out


In [79]:
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()


model = Autoencoder()


optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

criterion = nn.MSELoss()

In [80]:

training_epochs = 15
total_batch = len(mnist_train) // batch_size


for epoch in range(training_epochs):
    avg_cost = 0
    for i, (X, Y) in enumerate(train_loader):
        X = X.view(X.size(0), -1)
        optimizer.zero_grad()

        hypothesis = model(X)
        
        cost = criterion(hypothesis, X)

        cost.backward()

        optimizer.step()

        if i % 200 == 0:
            print("Epoch= {},\t batch = {},\t cost = {:.4f}".format(epoch, i, cost.item()))
        
        
            
        avg_cost += cost.data / total_batch

    
    pic = to_img(X.cpu().data)
    save_image(pic, './data/auto-enc/image_{}.png'.format(epoch))
        
    pic = to_img(hypothesis.cpu().data)
    save_image(pic, './data/auto-enc/mod-image_{}.png'.format(epoch))
    
    print("Epoch: {}, averaged cost = {:.4f}".format(epoch + 1, avg_cost.item()))


print('Learning Finished!')

Epoch= 0,	 batch = 0,	 cost = 0.1054
Epoch= 0,	 batch = 200,	 cost = 0.0148
Epoch: 1, averaged cost = 0.0258
Epoch= 1,	 batch = 0,	 cost = 0.0117
Epoch= 1,	 batch = 200,	 cost = 0.0104
Epoch: 2, averaged cost = 0.0110
Epoch= 2,	 batch = 0,	 cost = 0.0103
Epoch= 2,	 batch = 200,	 cost = 0.0088
Epoch: 3, averaged cost = 0.0094
Learning Finished!
